In [ ]:
import orsopy
import matplotlib.pyplot as plt

import scipp as sc
import sciline
from ess import amor
from ess.reflectometry.types import *
from ess.amor.types import ChopperPhase

## Recreate reference files using Jochens Amor data reduction software

#### step 1
```
%%bash
git clone https://github.com/jochenstahn/amor.git
wget -P raw https://public.esss.dk/groups/scipp/ess/amor/1/amor2023n000{608..614}.hdf
```

#### step 2
```
%%bash
args='-Y 2023 -d raw -n 614 -m 0.05 -co -7.5 -of ort'

rm raw/614.norm
python amor/neos.py $args -f 614

for i in `seq 608 613`
do
    python amor/neos.py $args -f $i -o $i $args 
done
```

## Run essreflectometry on the same files

In [ ]:
pl = sciline.Pipeline(
    (*amor.providers, *amor.data.providers),
    params=amor.default_parameters,
)

pl[SampleSize[Sample]] = sc.scalar(10., unit='mm')
pl[SampleSize[Reference]] = sc.scalar(10., unit='mm')

# In Jochens workflow:
#    * divergence mask: [-0.7, 0.7]
#    * note, divergence relative to beam center
pl[WavelengthBins] = sc.geomspace('wavelength', 2.8, 12.5, 300, unit='angstrom')
pl[YIndexLimits] =  sc.scalar(11, unit=None), sc.scalar(41, unit=None)
pl[ZIndexLimits] =  sc.scalar(80, unit=None), sc.scalar(370, unit=None)

# Chopper phase value in the file is wrong, so we set it manually
pl[ChopperPhase[Reference]] = sc.scalar(-7.5, unit='deg')
# The sample rotation value in the file is slightly off, so we set it manually
pl[SampleRotation[Reference]] = sc.scalar(0.65, unit='deg')
pl[PoochFilename[Reference]] = "amor2023n000614.hdf"

from ess.reflectometry import orso
for p in (*orso.providers, *amor.orso.providers):
    pl.insert(p)

pl[orso.OrsoCreator] = orso.OrsoCreator(
    orsopy.fileio.base.Person(
        name='Max Mustermann',
        affiliation='European Spallation Source ERIC',
        contact='max.mustermann@ess.eu',
    )
)
pl[orso.OrsoSample] = orsopy.fileio.data_source.Sample(
    name='Ni / Ti Multilayer',
    model=orsopy.fileio.data_source.SampleModel(
        stack='air | (Ni | Ti) * 5 | Si',
    ),
)

In [ ]:
from ess.reflectometry import save_reference, load_reference

pl[ReferenceFilePath] = save_reference(pl, './reference.hdf5')
pl.insert(load_reference)

In [ ]:
pl[QBins] = sc.geomspace(dim='Q', start=0.00505, stop=2.93164766e-01, num=391, unit='1/angstrom')

mu = {
    608: 0.8,
    609: 2.2,
    610: 3.6,
    611: 5.0,
    612: 0.6,
    613: 0.6,
}

for i in range(608, 614):
    print(i, '...', end='')
    pl[ChopperPhase[Sample]] = sc.scalar(-7.5, unit='deg')
    pl[SampleRotation[Sample]] = sc.scalar(mu[i] + 0.05, unit='deg')
    pl[PoochFilename[Sample]] = f"amor2023n000{i}.hdf"
    pl.compute(orso.OrsoIofQDataset).save(f'raw/ess-{i}.ort')
    print('Done!')


## Plot reflectivity curve comparison

In [ ]:
fig, axs = plt.subplots(3, 2, figsize=(12, 12))

for ax, i in zip(axs.ravel(), range(608, 614)):

    d = orsopy.fileio.load_orso(amor.data.get_path(f'{i}.Rqz.ort'))[0].data
    ax.errorbar(d[:, 0], d[:, 1], yerr=d[:, 2], label='PSI Amor software')

    d = orsopy.fileio.load_orso(f'raw/ess-{i}.ort')[0].data
    d[d[:, 1] == 0, 1:3] = float('nan')
    ax.errorbar(d[:, 0], d[:, 1], yerr=d[:, 2], label='ESS Reflectometry')
    ax.set_yscale('log')
    ax.set_ylim([1e-4, None])
    ax.legend()
    ax.set_title(f'amor2023n000{i}')
    ax.set_xlabel('Q [1/A]')
    ax.set_ylabel('Reflectivity')

plt.tight_layout()
plt.savefig('comparison.png')

## Overlay curves from same sample at different angle, should be on top of each other

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(12, 4))

for ax, facility, fname in zip(axs.ravel(), ('PSI Amor', 'ESSreflectometry'), ('{i}.Rqz.ort', 'raw/ess-{i}.ort')):
    for i in range(608, 612):

        d = orsopy.fileio.load_orso(amor.data.get_path(fname.format(i=i)) if facility.startswith('PSI') else fname.format(i=i))[0].data
        d[d[:, 1] == 0, 1:3] = float('nan')
        ax.errorbar(d[:, 0], d[:, 1], yerr=d[:, 2], label=f'{i}')
    
    ax.set_yscale('log')
    ax.set_ylim([1e-4, None])
    ax.legend()
    ax.set_title(facility)
    ax.set_xlabel('Q [1/A]')
    ax.set_ylabel('Reflectivity')
